# Hypothesis Testing in Python: A Guided Tutorial
 ---

This notebook is designed to provide a step-by-step guide to hypothesis testing
using three different distributions: Normal, Binomial, and Poisson.

The notebook will explain how hypothesis testing links to concepts of distributions and confidence intervals (CI).
For each distribution, we'll walk through a case study example.


In [1]:

# Import necessary libraries
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.weightstats import ztest
from scipy.stats import poisson, norm, t, binom

# Seed for reproducibility
np.random.seed(42)


---
## 1. Introduction to Hypothesis Testing

In this section, we introduce the general steps in hypothesis testing and link it with distributions and confidence intervals.

Steps in Hypothesis Testing:
1. State the null hypothesis (H0) and alternative hypothesis (H1).

2. Choose a significance level (α).

3. Collect and prepare the data.

4. Choose the appropriate test based on the distribution and sample size.

5. Conduct the test and calculate the test statistic.

6. Determine the p-value.

7. Make a decision: Reject or fail to reject the null hypothesis.

8. Interpret the results.

Key Concepts:

- Distributions help determine the appropriate hypothesis test.

- Confidence intervals can offer insights into hypothesis testing decisions.

---

# 2. Case Study 1: Normal Distribution

**Problem:**  
You are a quality control analyst at a factory producing light bulbs. 
The average lifetime of the light bulbs is expected to be 1000 hours.  

You take a sample of 50 bulbs and want to test if the mean lifetime is significantly different from 1000 hours.


In [2]:
sample_size = 50
population_mean = 1000  # Population mean
population_std = 100    # Population standard deviation (known)

# Simulate sample data (assuming a normal distribution)
sample_data = np.random.normal(loc = population_mean, 
                               scale = population_std, 
                               size = sample_size)

sample_data

array([1049.6714153 ,  986.17356988, 1064.76885381, 1152.30298564,
        976.58466253,  976.58630431, 1157.92128155, 1076.74347292,
        953.05256141, 1054.25600436,  953.65823072,  953.42702464,
       1024.19622716,  808.67197553,  827.50821675,  943.77124708,
        898.71688797, 1031.42473326,  909.19759245,  858.76962987,
       1146.56487689,  977.42236995, 1006.75282047,  857.52518138,
        945.56172755, 1011.09225897,  884.90064226, 1037.56980183,
        939.93613101,  970.83062502,  939.82933878, 1185.22781845,
        998.65027753,  894.2289071 , 1082.25449121,  877.915635  ,
       1020.8863595 ,  804.03298761,  867.18139511, 1019.68612359,
       1073.846658  , 1017.13682812,  988.43517176,  969.88963044,
        852.14780096,  928.01557916,  953.9361229 , 1105.71222262,
       1034.36182896,  823.69598446])

## Step 2: State Hypotheses
- $H_{0}$: μ = 1000 (Null Hypothesis: Mean lifetime is 1000 hours)
- $H_{1}$: μ ≠ 1000 (Alternative Hypothesis: Mean lifetime is different from 1000 hours)

## Step 3: Significance Level (alpha)
- The significance level, denoted as alpha (α), is set at 0.05 for this test. 

This means there is a 5% risk of rejecting the null hypothesis when it is actually true, which translates to a 95% confidence level in your results. If the p-value is below 0.05, you can conclude that the mean lifetime of the bulbs is significantly different from 1000 hours.

## Step 4: Calculate the sample mean and standard error

In [4]:
sample_mean = np.mean(sample_data)
standard_error = population_std / np.sqrt(sample_size)

print(sample_mean, standard_error)
print(population_mean, population_std)

977.4526094743861 14.14213562373095
1000 100


## Step 4: Perform the Z-test (since the population std is known)

<details><summary><font size=5, color="gold">Z-Test (Z-Score Test)</font></summary>

- **Purpose**: The z-test is used to compare a **sample mean** to a known population mean or to compare proportions.

- **Scenarios**:

  - When the population variance is **known**.
  - When dealing with large sample sizes.

- **Steps**:

  1. Calculate the sample mean.

  2. Compute the **z-score** using the formula:

     # $z = \frac{{\bar{x} - \mu}}{{\frac{{\sigma}}{{\sqrt{n}}}}}$

     

     where:

     - $\bar{x}$ is the sample mean.
     - $mu$ is the population mean.
     - $sigma$ is the population standard deviation.
     - $n$ is the sample size.

  3. Compare the z-score to the critical z-value based on the desired significance level.

- **Output**: A p-value indicating whether the sample mean is significantly different from the population mean
</details>

In [5]:
z_statistic = (sample_mean - population_mean) / standard_error
p_value = 2 * (1 - norm.cdf(abs(z_statistic))) # we are calculating the 

print(f"Sample Mean: {sample_mean}")
print(f"Z-Statistic: {z_statistic}")
print(f"P-Value: {p_value}")

#https://www.sjsu.edu/faculty/gerstman/StatPrimer/z-two-tails.pdf

Sample Mean: 977.4526094743861
Z-Statistic: -1.5943412738722889
P-Value: 0.11085961800021749


## Step 5: Decision Making

In [7]:
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis.")
else:
    print("Fail to reject the null hypothesis.")

Fail to reject the null hypothesis.


### Interpretation:
`If we reject the null hypothesis, it suggests that the mean lifetime of the bulbs differs significantly from 1000 hours.`
        
`If we fail to reject, we conclude that there is no significant difference from 1000 hours.`

---

# 3. Case Study 2: Binomial Distribution

**Problem:**  
You are testing a new drug. 

In clinical trials, the drug is expected to have a 60% success rate.  

You test the drug on 20 patients and observe 15 successes. 

Test if the success rate is significantly different from 60%.



## Step 1: Generate the sample data

In [9]:
n_trials = 20  # Number of trials
successes = 15  # Observed successes
success_rate_hypothesis = 0.60  # Hypothesized success rate

## Step 2: State Hypotheses
- $H_{0}$: p = 0.60 (Null Hypothesis: Success rate is 60%)
- $H_{1}$: p ≠ 0.60 (Alternative Hypothesis: Success rate is different from 60%)

## Step 3: Perform the Z-test for proportion

In [10]:
n_trials = 20  # Number of trials
successes = 15  # Observed successes
success_rate_hypothesis = 0.60  # Hypothesized success rate
z_stat, p_value = proportions_ztest(successes, n_trials, value=success_rate_hypothesis)

print(f"Z-Statistic: {z_stat}")
print(f"P-Value: {p_value}")

Z-Statistic: 1.549193338482967
P-Value: 0.12133525035848204


<details><summary><font size=5, color="gold"><code>proportions_ztest()</code> function</font></summary>

The proportions_ztest function calculates the z-statistic and p-value for a z-test comparing the proportion of successes in a sample to a hypothesized success rate. Here’s how it works:

**Z-statistic formula:**   
### $$ z = \frac{\hat{p} - p_0}{\sqrt{\frac{p_0 (1 - p_0)}{n}}} $$

**Where:**

$\hat{p}$ is the sample proportion (successes/n_trials).

$p_0$ is the hypothesized success rate.

$n$ is the sample size.

**P-value calculation:** The p-value is determined by finding the probability that the z-statistic falls within the distribution, adjusted for the type of test (two-tailed, one-tailed).
</detail>


In [13]:
# applying the formula and calculating the p_value
z_calc = ((successes/n_trials)-success_rate_hypothesis) / (
    (
        np.sqrt(
        (success_rate_hypothesis*(1-success_rate_hypothesis))/n_trials
        )
    )
)
p_value = 2 * (1 - norm.cdf(z_calc))

z_calc, p_value

(1.3693063937629155, 0.17090352023079736)

## Step 4: Decision Making


In [14]:
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis.")
else:
    print("Fail to reject the null hypothesis.")

Fail to reject the null hypothesis.


### Interpretation:
`If we reject the null hypothesis, it suggests that the success rate of the drug differs significantly from 60%.`

`If we fail to reject, we conclude that there is no significant difference from 60%.`

---

# 4. Case Study 3: Poisson Distribution

**Problem:**  
You work for a customer support center.  
On average, 10 calls come in per hour.  
You want to test if the average number of calls per hour has changed. 
You observe 8 calls in the last hour. 



## Step 1: State Hypotheses
- $H_{0}$: λ = 10 (Null Hypothesis: Average number of calls is 10 per hour)
- $H_{1}$: λ ≠ 10 (Alternative Hypothesis: Average number of calls is different from 10 per hour)

## Step 2: Calculate the test statistic



In [15]:
#For Poisson distribution, the test statistic is based on the observed count and the expected count.
observed_calls = 8
expected_calls = 10

## Step 3: Calculate the p-value using the Poisson distribution

In [ ]:
# Since it's a two-tailed test, we calculate the cumulative probability on both sides of the observed value
p_value = 2 * min(poisson.cdf(expected_calls, observed_calls), 1 - poisson.cdf(observed_calls - 1, expected_calls))

print(f"Observed Calls: {observed_calls}")
print(f"Expected Calls: {expected_calls}")
print(f"P-Value: {p_value}")



## Step 4: Decision Making

In [ ]:
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis.")
else:
    print("Fail to reject the null hypothesis.")

### Interpretation:
`If we reject the null hypothesis, it suggests that the average number of calls per hour has changed significantly from 10.`

`If we fail to reject, we conclude that there is no significant difference from 10 calls per hour.`

---
# Conclusion

In this notebook, we've explored hypothesis testing using three different distributions: Normal, Binomial, and Poisson.

We've linked these tests to the concepts of distributions and confidence intervals.

Remember that hypothesis testing is a powerful tool for making decisions based on sample data,
and it's essential to choose the appropriate test based on your data's distribution and characteristics.


